### ### Build the base panel (50 states × month)

In [14]:
# 1) State x Month panel
import pandas as pd

STATE_FIPS_50 = [
    "01","02","04","05","06","08","09","10","12","13","15","16","17","18","19","20",
    "21","22","23","24","25","26","27","28","29","30","31","32","33","34","35","36",
    "37","38","39","40","41","42","44","45","46","47","48","49","50","51","53","54","55","56"
]

def build_state_month_panel(start="2005-01-01", end="2024-12-01"):
    months = pd.date_range(start=start, end=end, freq="MS")  # Month Start
    panel = (
        pd.MultiIndex.from_product([STATE_FIPS_50, months], names=["state_fips", "date"])
          .to_frame(index=False)
    )
    return panel

panel = build_state_month_panel("2005-01-01", "2024-12-01")
print(panel.shape)  # ~ (12000, 2)
panel.head()

(12000, 2)


,state_fips,date
0,01,2005-01-01
1,01,2005-02-01
2,01,2005-03-01
3,01,2005-04-01
4,01,2005-05-01


In [16]:
# 2) FRED pull (robust for daily/weekly/monthly)
#    - DO NOT force "frequency=m" (some series are daily/weekly; we aggregate ourselves)
import os
import requests
from dotenv import load_dotenv

load_dotenv()
FRED_API_KEY = os.getenv("FRED_API_KEY")
if not FRED_API_KEY:
    raise ValueError("FRED_API_KEY not found. Ensure it's set in your .env and load_dotenv() is called.")

def fetch_fred_to_monthly(series_id, start="2005-01-01", end="2024-12-31", agg="last"):
    """
    Fetch a FRED series and aggregate to monthly at YYYY-MM-01 (month start).

    agg:
      - 'last': last available observation in the month (good for rates/yields)
      - 'mean': monthly average (sometimes better for volatile daily series)
    """
    url = "https://api.stlouisfed.org/fred/series/observations"
    params = {
        "series_id": series_id,
        "api_key": FRED_API_KEY,
        "file_type": "json",
        "observation_start": start,
        "observation_end": end,
    }

    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()

    data = r.json()
    if "observations" not in data:
        raise ValueError(f"No 'observations' in response for series_id={series_id}. Response keys: {list(data.keys())}")

    obs = data["observations"]
    df = pd.DataFrame(obs)[["date", "value"]]

    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    # Convert to numeric; FRED may return "." for missing values
    df["value"] = pd.to_numeric(df["value"].replace(".", pd.NA), errors="coerce")

    # Monthly bucket at month start (YYYY-MM-01)
    df["month"] = df["date"].values.astype("datetime64[M]")

    if agg == "mean":
        out = df.groupby("month", as_index=False)["value"].mean()
    else:
        # last non-null in the month
        out = (
            df.sort_values("date")
              .groupby("month", as_index=False)["value"]
              .last()
        )

    out = out.rename(columns={"month": "date", "value": series_id}).sort_values("date")
    return out


In [17]:

# 3) Expanded FRED feature list (more complete)
#    Notes:
#    - Some are monthly already; some are daily/weekly but handled by the function above.
#    - Start with ~25–35 series; add more later if needed.
fred_ids = [
    # --- Labor market ---
    "UNRATE",      # Unemployment rate (monthly)
    "PAYEMS",      # Nonfarm payrolls (monthly)
    "CIVPART",     # Labor force participation (monthly)
    "EMRATIO",     # Employment-population ratio (monthly)
    "U6RATE",      # Unemployment (U-6) (monthly)
    "AWHMAN",      # Avg weekly hours, manufacturing (monthly)
    "AHETPI",      # Avg hourly earnings, total private (monthly)

    # --- Inflation / prices ---
    "CPIAUCSL",    # CPI (monthly)
    "CPILFESL",    # Core CPI (monthly)
    "PCEPI",       # PCE price index (monthly)
    "PCEPILFE",    # Core PCE (monthly)
    "PPIFDG",      # PPI Final Demand (monthly) (if unavailable, remove)

    # --- Output / demand ---
    "INDPRO",      # Industrial production (monthly)
    "RRSFS",       # Retail sales (monthly)
    "DGORDER",     # Durable goods orders (monthly) (if unavailable, remove)
    "UMCSENT",     # Consumer sentiment (monthly)
    "NAPM",        # ISM PMI (monthly) (if unavailable, remove)

    # --- Housing ---
    "HOUST",       # Housing starts (monthly)
    "PERMIT",      # Building permits (monthly)
    "CSUSHPINSA",  # Case-Shiller US HPI (monthly)
    "MSPUS",       # Median sales price of houses (quarterly; will upsample to monthly)

    # --- Rates / yields (often daily/weekly; we aggregate) ---
    "FEDFUNDS",    # Fed funds (monthly)
    "DGS3MO",      # 3M Treasury (daily)
    "DGS2",        # 2Y Treasury (daily)
    "DGS10",       # 10Y Treasury (daily)
    "MORTGAGE30US",# 30Y mortgage rate (weekly)

    # --- Financial conditions / spreads / risk ---
    "BAA",         # Moody's Seasoned Baa Corporate Bond Yield (monthly)
    "AAA",         # Moody's Seasoned Aaa Corporate Bond Yield (monthly)
    "BAMLH0A0HYM2",# ICE BofA US High Yield OAS (daily) (if unavailable, remove)
    "NFCI",        # Chicago Fed National Financial Conditions Index (weekly)

    # --- Money / credit aggregates ---
    "M2SL",        # M2 money stock (weekly)
    "WALCL",       # Fed balance sheet total assets (weekly)
]

# Optional: If you want a leaner set first, comment out a few of the “maybe unavailable” ones:
# "PPIFDG", "DGORDER", "NAPM", "BAMLH0A0HYM2", "NFCI", "MSPUS"

In [19]:
# =========================
# 4) Pull & merge all FRED series into one wide monthly table
# =========================
start_date = "2005-01-01"
end_date   = "2024-12-31"

fred_wide = None
failed = []

for sid in fred_ids:
    try:
        # For yields/spreads and many rates, 'last' is fine; you can switch to mean for daily series if you prefer.
        s = fetch_fred_to_monthly(sid, start=start_date, end=end_date, agg="last")
        fred_wide = s if fred_wide is None else fred_wide.merge(s, on="date", how="outer")
    except Exception as e:
        failed.append((sid, str(e)))

fred_wide = fred_wide.sort_values("date").reset_index(drop=True)

print("FRED wide shape:", fred_wide.shape)
if failed:
    print("\nSeries that failed (you can remove/replace these IDs):")
    for sid, err in failed:
        print(f"- {sid}: {err}")

fred_wide.tail()

FRED wide shape: (240, 32)

Series that failed (you can remove/replace these IDs):
- NAPM: 400 Client Error: Bad Request for url: https://api.stlouisfed.org/fred/series/observations?series_id=NAPM&api_key=f07764a2df84ceb3f26f8b0545012e52&file_type=json&observation_start=2005-01-01&observation_end=2024-12-31


,date,UNRATE,PAYEMS,CIVPART,EMRATIO,U6RATE,AWHMAN,AHETPI,CPIAUCSL,CPILFESL,...,DGS3MO,DGS2,DGS10,MORTGAGE30US,BAA,AAA,BAMLH0A0HYM2,NFCI,M2SL,WALCL
235,2024-08-01,4.2,158074,62.7,60.0,7.8,40.7,30.27,314.131,319.839,...,5.21,3.91,3.91,6.35,5.60,4.87,3.17,-0.39378,21171.0,7123238.0
236,2024-09-01,4.1,158314,62.7,60.1,7.7,40.7,30.38,314.851,320.835,...,4.73,3.66,3.81,6.08,5.42,4.68,3.03,-0.42880,21257.4,7080059.0
237,2024-10-01,4.1,158358,62.5,60.0,7.7,40.5,30.49,315.564,321.688,...,4.64,4.16,4.28,6.72,5.63,4.95,2.88,-0.44461,21308.1,7013490.0
238,2024-11-01,4.2,158619,62.4,59.8,7.7,40.7,30.58,316.449,322.619,...,4.58,4.13,4.18,6.81,5.78,5.14,2.74,-0.47549,21408.0,6905140.0
239,2024-12-01,4.1,158942,62.5,59.9,7.6,40.9,30.67,317.603,323.296,...,4.37,4.25,4.58,6.85,5.80,5.20,2.92,-0.48552,21424.5,6885963.0


In [20]:
# =========================
# 5) Merge into your state-month panel (replicates national FRED features across states)
# =========================
state_month = panel.merge(fred_wide, on="date", how="left")
print("State-month table shape:", state_month.shape)
state_month.head()

State-month table shape: (12000, 33)


,state_fips,date,UNRATE,PAYEMS,CIVPART,EMRATIO,U6RATE,AWHMAN,AHETPI,CPIAUCSL,...,DGS3MO,DGS2,DGS10,MORTGAGE30US,BAA,AAA,BAMLH0A0HYM2,NFCI,M2SL,WALCL
0,01,2005-01-01,5.3,132781,65.8,62.4,9.2,40.7,15.90,191.6,...,2.51,3.29,4.14,5.66,6.02,5.36,3.29,-0.70536,6430.8,807262.0
1,01,2005-02-01,5.4,133033,65.9,62.4,9.2,40.7,15.93,192.4,...,2.76,3.59,4.36,5.69,5.82,5.20,2.83,-0.71276,6438.9,804576.0
2,01,2005-03-01,5.2,133152,65.9,62.4,9.1,40.4,15.97,193.1,...,2.79,3.80,4.50,6.04,6.06,5.40,3.52,-0.66764,6448.3,807551.0
3,01,2005-04-01,5.2,133519,66.1,62.7,9.0,40.4,16.01,193.7,...,2.90,3.66,4.21,5.78,6.05,5.33,4.19,-0.61154,6461.8,809797.0
4,01,2005-05-01,5.1,133689,66.1,62.8,8.9,40.4,16.03,193.6,...,2.99,3.60,4.00,5.65,6.01,5.15,4.13,-0.60139,6479.0,810160.0


In [21]:
# =========================
# 6) (Highly recommended) Add a few engineered features from FRED (optional now)
#    Example: yield spreads and YoY inflation, etc.
# =========================
# Yield curve spreads (if all series present)
if {"DGS10", "DGS2", "DGS3MO"}.issubset(state_month.columns):
    state_month["SPREAD_10Y_2Y"] = state_month["DGS10"] - state_month["DGS2"]
    state_month["SPREAD_10Y_3M"] = state_month["DGS10"] - state_month["DGS3MO"]

# YoY inflation from CPI (replicated across states; still useful)
if "CPIAUCSL" in state_month.columns:
    # Compute YoY on the date dimension, then merge back (safer than groupby on states)
    tmp = state_month[["date", "CPIAUCSL"]].drop_duplicates("date").sort_values("date")
    tmp["CPI_YOY"] = tmp["CPIAUCSL"] / tmp["CPIAUCSL"].shift(12) - 1
    state_month = state_month.merge(tmp[["date", "CPI_YOY"]], on="date", how="left")

state_month.tail()

,state_fips,date,UNRATE,PAYEMS,CIVPART,EMRATIO,U6RATE,AWHMAN,AHETPI,CPIAUCSL,...,MORTGAGE30US,BAA,AAA,BAMLH0A0HYM2,NFCI,M2SL,WALCL,SPREAD_10Y_2Y,SPREAD_10Y_3M,CPI_YOY
11995,56,2024-08-01,4.2,158074,62.7,60.0,7.8,40.7,30.27,314.131,...,6.35,5.60,4.87,3.17,-0.39378,21171.0,7123238.0,0.00,-1.30,0.026109
11996,56,2024-09-01,4.1,158314,62.7,60.1,7.7,40.7,30.38,314.851,...,6.08,5.42,4.68,3.03,-0.42880,21257.4,7080059.0,0.15,-0.92,0.024325
11997,56,2024-10-01,4.1,158358,62.5,60.0,7.7,40.5,30.49,315.564,...,6.72,5.63,4.95,2.88,-0.44461,21308.1,7013490.0,0.12,-0.36,0.025714
11998,56,2024-11-01,4.2,158619,62.4,59.8,7.7,40.7,30.58,316.449,...,6.81,5.78,5.14,2.74,-0.47549,21408.0,6905140.0,0.05,-0.40,0.027142
11999,56,2024-12-01,4.1,158942,62.5,59.9,7.6,40.9,30.67,317.603,...,6.85,5.80,5.20,2.92,-0.48552,21424.5,6885963.0,0.33,0.21,0.028724


### ACS pull function (by state, by year)

In [22]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv

load_dotenv()
CENSUS_API_KEY = os.getenv("CENSUS_API_KEY")
if not CENSUS_API_KEY:
    raise ValueError("CENSUS_API_KEY not found. Ensure it's set in your .env and load_dotenv() is called.")

def fetch_acs_state_year(year: int, variables: list[str], dataset: str = "acs/acs5") -> pd.DataFrame:
    """
    Fetch ACS data for all states for a given year.
    Returns: state_fips, state_name, year, <variables...>
    """
    url = f"https://api.census.gov/data/{year}/{dataset}"
    params = {
        "get": ",".join(["NAME"] + variables),
        "for": "state:*",
        "key": CENSUS_API_KEY
    }

    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    data = r.json()

    cols = data[0]
    rows = data[1:]
    df = pd.DataFrame(rows, columns=cols)

    df = df.rename(columns={"state": "state_fips", "NAME": "state_name"})
    df["year"] = year

    # Convert variables to numeric
    for v in variables:
        df[v] = pd.to_numeric(df[v], errors="coerce")

    return df[["state_fips", "state_name", "year"] + variables]

In [34]:
# Pick a starter ACS variable list (by state)
acs_vars = [
    "B19013_001E",  # Median household income
    "B01003_001E",  # Total population
    "B25077_001E",  # Median home value
    "B25064_001E",  # Median gross rent
    "B17001_002E",  # Poverty count (below poverty)
]

In [ ]:
# Pull ACS annual for valid years (ACS5 starts 2010+)
start_year = 2010
end_year = 2023
years = list(range(start_year, end_year + 1))

acs_annual = pd.concat(
    [fetch_acs_state_year(y, acs_vars, dataset="acs/acs5") for y in years],
    ignore_index=True
)

print("ACS annual shape:", acs_annual.shape)
acs_annual.head()

ACS annual shape: (728, 8)


,state_fips,state_name,year,B19013_001E,B01003_001E,B25077_001E,B25064_001E,B17001_002E
0,01,Alabama,2010,42081,4712651,117600,644,786544
1,02,Alaska,2010,66521,691189,229100,972,64245
2,04,Arizona,2010,50448,6246816,215000,856,933113
3,05,Arkansas,2010,39267,2872684,102300,617,502684
4,06,California,2010,60883,36637290,458500,1147,4919945


In [42]:
import pandas as pd

def expand_acs_annual_to_monthly(
    acs_annual: pd.DataFrame,
    start_month="2005-01-01",
    end_month="2024-12-01",
    vars_to_fill=None,
    backfill_before_first_year=False
) -> pd.DataFrame:
    """
    Expand annual ACS values to monthly by state using forward fill.

    - ACS is annual; we anchor each year at YYYY-01-01
    - Then create monthly dates and forward-fill within each state
    - Optionally backfill months before the first ACS year using the first available value
    """
    if vars_to_fill is None:
        vars_to_fill = [c for c in acs_annual.columns if c not in ["state_fips", "state_name", "year"]]

    acs = acs_annual.copy()

    # Year -> month-start anchor (Jan of that year)
    acs["date"] = pd.to_datetime(acs["year"].astype(str) + "-01-01").values.astype("datetime64[M]")

    monthly_index = pd.DataFrame({"date": pd.date_range(start_month, end_month, freq="MS")})

    parts = []
    for state, g in acs.groupby("state_fips"):
        g = g.sort_values("date").merge(monthly_index, on="date", how="right")
        g["state_fips"] = state
        g["state_name"] = g["state_name"].ffill()

        for v in vars_to_fill:
            g[v] = g[v].ffill()
            if backfill_before_first_year:
                g[v] = g[v].bfill()

        parts.append(g[["state_fips", "state_name", "date"] + vars_to_fill])

    return pd.concat(parts, ignore_index=True)

In [44]:
acs_monthly = expand_acs_annual_to_monthly(
    acs_annual,
    start_month="2005-01-01",
    end_month="2024-12-01",
    vars_to_fill=acs_vars,
    backfill_before_first_year=True   # ✅ fills 2005–2009 using 2010
)

print("ACS monthly shape:", acs_monthly.shape)

# Sanity checks
print(acs_monthly[(acs_monthly["state_fips"]=="01") & (acs_monthly["date"]<"2010-01-01")][acs_vars].head())
print(acs_monthly[(acs_monthly["state_fips"]=="01") & (acs_monthly["date"]>="2023-01-01")][acs_vars].tail())

ACS monthly shape: (12480, 8)
   B19013_001E  B01003_001E  B25077_001E  B25064_001E  B17001_002E
0      42081.0    4712651.0     117600.0        644.0     786544.0
1      42081.0    4712651.0     117600.0        644.0     786544.0
2      42081.0    4712651.0     117600.0        644.0     786544.0
3      42081.0    4712651.0     117600.0        644.0     786544.0
4      42081.0    4712651.0     117600.0        644.0     786544.0
     B19013_001E  B01003_001E  B25077_001E  B25064_001E  B17001_002E
235      62027.0    5054253.0     195100.0        963.0     768185.0
236      62027.0    5054253.0     195100.0        963.0     768185.0
237      62027.0    5054253.0     195100.0        963.0     768185.0
238      62027.0    5054253.0     195100.0        963.0     768185.0
239      62027.0    5054253.0     195100.0        963.0     768185.0


### Merge ACS with Fred table on state fips and date key columns

In [47]:
state_month_full = state_month.merge(
    acs_monthly,
    on=["state_fips", "date"],
    how="left"
)

print(state_month_full.shape)
print(state_month_full.head(50))

(12000, 42)
   state_fips       date  UNRATE  PAYEMS  CIVPART  EMRATIO  U6RATE  AWHMAN  \
0          01 2005-01-01     5.3  132781     65.8     62.4     9.2    40.7   
1          01 2005-02-01     5.4  133033     65.9     62.4     9.2    40.7   
2          01 2005-03-01     5.2  133152     65.9     62.4     9.1    40.4   
3          01 2005-04-01     5.2  133519     66.1     62.7     9.0    40.4   
4          01 2005-05-01     5.1  133689     66.1     62.8     8.9    40.4   
5          01 2005-06-01     5.0  133947     66.1     62.7     9.0    40.4   
6          01 2005-07-01     5.0  134300     66.1     62.8     8.8    40.5   
7          01 2005-08-01     4.9  134503     66.2     62.9     8.9    40.5   
8          01 2005-09-01     5.0  134554     66.1     62.8     9.0    40.7   
9          01 2005-10-01     5.0  134643     66.1     62.8     8.7    41.0   
10         01 2005-11-01     5.0  134999     66.0     62.7     8.7    40.9   
11         01 2005-12-01     4.9  135158     66.0   

In [48]:
import os

os.makedirs("../data", exist_ok=True)
out_path = os.path.join("../data", "state_month_full.csv")

state_month_full.to_csv(out_path, index=False)
print("Saved:", out_path)

Saved: ../data/state_month_full.csv


Among above features UNRATE, NFCI, SPREAD_10Y_2Y  or SPREAD_10Y_3M can be picked as target variables. I prefer to go with NFCI is explicitly a broad financial stress/conditions index (credit, leverage, funding/liquidity signals). That maps most directly to “tightening conditions → higher risk → larger capital buffers.”